### Data loading

In [1]:
// Load wrangled datasets from Phase 2: part1.csv; part2.csv; part3.csv

val creditRiskdf1 = spark.read.option("header","true").csv("part1.csv")
val creditRiskdf2 = spark.read.option("header","true").csv("part2.csv")
val creditRiskdf3 = spark.read.option("header","true").csv("part3.csv")

Intitializing Scala interpreter ...

Spark Web UI available at http://b2cc1a63ffdb:4043
SparkContext available as 'sc' (version = 2.4.3, master = local[*], app id = local-1559458223502)
SparkSession available as 'spark'


2019-06-02 06:50:19,624 WARN  [main] util.NativeCodeLoader (NativeCodeLoader.java:<clinit>(62)) - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2019-06-02 06:50:23,017 WARN  [Thread-4] util.Utils (Logging.scala:logWarning(66)) - Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
2019-06-02 06:50:23,021 WARN  [Thread-4] util.Utils (Logging.scala:logWarning(66)) - Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
2019-06-02 06:50:23,022 WARN  [Thread-4] util.Utils (Logging.scala:logWarning(66)) - Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


creditRiskdf1: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 283 more fields]
creditRiskdf2: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 283 more fields]
creditRiskdf3: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 283 more fields]


In [2]:
// combine the 3 dataframes to 1
val creditRiskdf1and2 = creditRiskdf1.union(creditRiskdf2)
val creditRiskdf = creditRiskdf1and2.union(creditRiskdf3)

creditRiskdf1and2: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 283 more fields]
creditRiskdf: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 283 more fields]


In [5]:
// check that the rows have been aggregated correctly

println("Total data rows in loaded dataframes:"+(
    creditRiskdf1.count()+creditRiskdf2.count()+creditRiskdf3.count()))

println("Total data rows in combined dataframe:"+creditRiskdf.count())

2019-06-02 06:53:09,570 WARN  [Thread-4] util.Utils (Logging.scala:logWarning(66)) - Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.debug.maxToStringFields' in SparkEnv.conf.
Total data rows in loaded dataframes:183875
Total data rows in combined dataframe:183875


### Data cleansing and reshaping

In [11]:
// Drop features with inconsistent values from dataset.

val incon_droplist = List("previous_loans_DAYS_FIRST_DUE_mean",
                      "previous_loans_SELLERPLACE_AREA_min",
                      "previous_loans_DAYS_FIRST_DUE_sum")

val df = creditRiskdf.drop(incon_droplist:_*)

incon_droplist: List[String] = List(previous_loans_DAYS_FIRST_DUE_mean, previous_loans_SELLERPLACE_AREA_min, previous_loans_DAYS_FIRST_DUE_sum)
df: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 280 more fields]
res1: df.type = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 280 more fields]


In [14]:
val creditRiskdf = df

creditRiskdf: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 280 more fields]


In [15]:
//string array of features
val creditRiskFeatures = creditRiskdf.columns

creditRiskFeatures: Array[String] = Array(EXT_SOURCE_2, EXT_SOURCE_3, client_installments_AMT_PAYMENT_min_sum, DAYS_BIRTH, AMT_CREDIT, AMT_ANNUITY, DAYS_EMPLOYED, bureau_DAYS_CREDIT_ENDDATE_max, bureau_DAYS_CREDIT_max, DAYS_ID_PUBLISH, bureau_DAYS_ENDDATE_FACT_max, bureau_AMT_CREDIT_SUM_DEBT_mean, previous_loans_CNT_PAYMENT_mean, client_cash_CNT_INSTALMENT_FUTURE_min_max, previous_loans_SELLERPLACE_AREA_max, DAYS_LAST_PHONE_CHANGE, bureau_AMT_CREDIT_SUM_max, bureau_AMT_CREDIT_SUM_mean, DAYS_REGISTRATION, client_installments_DAYS_INSTALMENT_max_max, previous_loans_AMT_DOWN_PAYMENT_max, CODE_GENDER_F, REGION_POPULATION_RELATIVE, client_cash_CNT_INSTALMENT_max_mean, bureau_DAYS_CREDIT_UPDATE_max, bureau_AMT_CREDIT_SUM_sum, client_installments_NUM_INSTALMENT_VERSION_mean_max, bureau_DAYS_CR...

In [36]:
creditRiskdf.cache()

2019-06-02 07:31:50,601 WARN  [Thread-4] execution.CacheManager (Logging.scala:logWarning(66)) - Asked to cache already cached data.


res14: creditRiskdf.type = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 280 more fields]


In [38]:
//write function to check if there are any outliers in a given domain and show boxplot [Sean]
//slice a small test list to ensure functionality
//var test_lis = creditRiskFeatures.slice(0,2) //TEST var for codeblock

var feat_list = List() //empty list to append features containing outliers in
for (feature <- creditRiskFeatures) {
    //summarise column and query the DataFrame output
    var firstQ = creditRiskdf.select(feature).summary().where(
        $"summary" === "25%").select(feature).first().mkString.toFloat
    var thirdQ = creditRiskdf.select(feature).summary().where(
        $"summary" === "75%").select(feature).first().mkString.toFloat
    //use formulas below to test threshold of outliers
    var testValHigh = thirdQ + (1.5 * thirdQ - firstQ)
    var testValLow = firstQ - (1.5 * thirdQ - firstQ)
    //check to see if thresholds are exceeded in the column and count
    var outHigh = creditRiskdf.filter(col(feature) > lit(testValHigh)).count()
    var outLow = creditRiskdf.filter(col(feature) < lit(testValLow)).count()

    //notify us whether or not column contains outliers
    if (outHigh > 0 || outLow > 0 ){
        println(feature.concat(": Contains outliers, #").concat((
            outHigh + outLow).toString))
        var feat_list = List.concat(feat_list, feature)
    }else if (outHigh = 0 || outLow = 0){
        println(feature.concat(": Does not contain outliers"))
    }
}

<console>: 52: error: recursive variable feat_list needs type

In [ ]:
//code block takes features containing outliers and filters main dataframe against them
//code will generate a subset of main dataset without outliers.
val clean_df = creditriskdf
//input feature list conatining features with outliers persisting
for (feature <- feat_list){
    //summarise column and query the DataFrame output
    var firstQ = creditRiskdf.select(feature).summary().where($"summary" === "25%").select(feature).first().mkString.toFloat
    var thirdQ = creditRiskdf.select(feature).summary().where($"summary" === "75%").select(feature).first().mkString.toFloat
    //use formulas below to test threshold of outliers
    var testValHigh = thirdQ + (1.5 * thirdQ - firstQ)
    var testValLow = firstQ - (1.5 * thirdQ - firstQ)

    val clean_df = clean_df.filter(feature < lit(testValHigh))
    val clean_df = clean_df.filter(feature > lit(testValLow))
}

### Machine learning

In [31]:
// random split of cleansed dataset to training set (70%), testing set (20%)
// and validation set (10%)

val Array(train_df,test_df,validation_df)=creditRiskdf.randomSplit(
    Array(0.7,0.2,0.1))

train_df: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 280 more fields]
test_df: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 280 more fields]
validation_df: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 280 more fields]


In [35]:
// cache train_df dataset for ML stage
train_df.cache()

res13: train_df.type = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 280 more fields]
